## Validate protein mutations with endstate corrections!!!

this should tell us if we are at all capable of closing cycles...

In [1]:
#!/usr/bin/env python
# coding: utf-8

# # Here, I document an attempt to validate a small set of protein mutations in vacuum and solvent with the following checks...

# 1. generate alanine dipeptide --> valine dipeptide in vac/solvent and conduct a forward _and_ reverse parallel tempering FEP calculation; the check passes if the forward free energy is equal to the reverse free energy within an error tolerance
# 2. generate alanine dipeptide --> valine dipeptide --> isoleucine dipeptide --> glycine dipeptide and attempt to close the thermodynamic cycle within an error tolerance

# In[ ]:


from __future__ import absolute_import

import networkx as nx
from perses.dispersed import feptasks
from perses.utils.openeye import *
from perses.utils.data import load_smi
from perses.annihilation.relative import HybridTopologyFactory
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
from perses.rjmc.topology_proposal import TopologyProposal, TwoMoleculeSetProposalEngine, SystemGenerator,SmallMoleculeSetProposalEngine, PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
import simtk.openmm.app as app
import sys

from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState

import pymbar
import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import copy
import pickle
import mdtraj as md
from io import StringIO
from openmmtools.constants import kB
import logging
import os
import dask.distributed as distributed
import parmed as pm
from collections import namedtuple
from typing import List, Tuple, Union, NamedTuple
from collections import namedtuple
import random
#beta = 1.0/(kB*temperature)
import itertools
import os
from openeye import oechem
from perses.utils.smallmolecules import render_atom_mapping
from perses.tests.utils import validate_endstate_energies

ENERGY_THRESHOLD = 1e-6
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
from perses.tests.utils import validate_endstate_energies


# In[ ]:


from perses.samplers.multistate import HybridSAMSSampler, HybridRepexSampler
from openmmtools.multistate import MultiStateReporter, MultiStateSamplerAnalyzer
from openmmtools import mcmc, utils
from perses.annihilation.lambda_protocol import LambdaProtocol


# In[ ]:


def generate_atp(phase = 'vacuum'):
    """
    modify the AlanineDipeptideVacuum test system to be parametrized with amber14ffsb in vac or solvent (tip3p)
    """
    import openmmtools.testsystems as ts
    atp = ts.AlanineDipeptideVacuum(constraints = app.HBonds, hydrogenMass = 4 * unit.amus)

    forcefield_files = ['gaff.xml', 'amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
    
    if phase == 'vacuum':
        barostat = None
        system_generator = SystemGenerator(forcefield_files,
                                       barostat = barostat,
                                       forcefield_kwargs = {'removeCMMotion': False, 
                                                            'ewaldErrorTolerance': 1e-4, 
                                                            'nonbondedMethod': app.NoCutoff,
                                                            'constraints' : app.HBonds, 
                                                            'hydrogenMass' : 4 * unit.amus})
        atp.system = system_generator.build_system(atp.topology) #update the parametrization scheme to amberff14sb
        
    elif phase == 'solvent':
        barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
        system_generator = SystemGenerator(forcefield_files,
                                   barostat = barostat,
                                   forcefield_kwargs = {'removeCMMotion': False, 
                                                        'ewaldErrorTolerance': 1e-4, 
                                                        'nonbondedMethod': app.PME,
                                                        'constraints' : app.HBonds, 
                                                        'hydrogenMass' : 4 * unit.amus})
    
    if phase == 'solvent':
        modeller = app.Modeller(atp.topology, atp.positions)
        modeller.addSolvent(system_generator._forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
        solvated_topology = modeller.getTopology()
        solvated_positions = modeller.getPositions()

        # canonicalize the solvated positions: turn tuples into np.array
        atp.positions = unit.quantity.Quantity(value = np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit = unit.nanometers)
        atp.topology = solvated_topology

        atp.system = system_generator.build_system(atp.topology)
    
    
    return atp, system_generator


# In[ ]:


def generate_top_pos_sys(topology, new_res, system, positions, system_generator):
    """generate point mutation engine, geometry_engine, and conduct topology proposal, geometry propsal, and hybrid factory generation"""
    #create the point mutation engine
    print(f"generating point mutation engine")
    point_mutation_engine = PointMutationEngine(wildtype_topology = topology,
                                                system_generator = system_generator,
                                                chain_id = '1', #denote the chain id allowed to mutate (it's always a string variable)
                                                max_point_mutants = 1,
                                                residues_allowed_to_mutate = ['2'], #the residue ids allowed to mutate
                                                allowed_mutations = [('2', new_res)], #the residue ids allowed to mutate with the three-letter code allowed to change
                                                aggregate = True) #always allow aggregation

    #create a geometry engine
    print(f"generating geometry engine")
    geometry_engine = FFAllAngleGeometryEngine(metadata=None, 
                                           use_sterics=False, 
                                           n_bond_divisions=100, 
                                           n_angle_divisions=180, 
                                           n_torsion_divisions=360, 
                                           verbose=True, 
                                           storage=None, 
                                           bond_softening_constant=1.0, 
                                           angle_softening_constant=1.0, 
                                           neglect_angles = False, 
                                           use_14_nonbondeds = True)

    #create a top proposal
    print(f"making topology proposal")
    topology_proposal, local_map_stereo_sidechain, new_oemol_sidechain, old_oemol_sidechain = point_mutation_engine.propose(current_system = system,
                                  current_topology = topology)

    #make a geometry proposal forward
    print(f"making geometry proposal from {list(topology.residues())[1].name} to {new_res}")
    forward_new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta)
    logp_reverse = geometry_engine.logp_reverse(topology_proposal, forward_new_positions, positions, beta)


    #create a hybrid topology factory
    f"making forward hybridtopologyfactory"
    forward_htf = HybridTopologyFactory(topology_proposal = topology_proposal,
                 current_positions =  positions,
                 new_positions = forward_new_positions,
                 use_dispersion_correction = False,
                 functions=None,
                 softcore_alpha = None,
                 bond_softening_constant=1.0,
                 angle_softening_constant=1.0,
                 soften_only_new = False,
                 neglected_new_angle_terms = [],
                 neglected_old_angle_terms = [],
                 softcore_LJ_v2 = True,
                 softcore_electrostatics = True,
                 softcore_LJ_v2_alpha = 0.85,
                 softcore_electrostatics_alpha = 0.3,
                 softcore_sigma_Q = 1.0,
                 interpolate_old_and_new_14s = False,
                 omitted_terms = None)
    
    if not topology_proposal.unique_new_atoms:
        assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
        assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
        vacuum_added_valence_energy = 0.0
    else:
        added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

    if not topology_proposal.unique_old_atoms:
        assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
        assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
        subtracted_valence_energy = 0.0
    else:
        subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential

#     self._vacuum_forward_neglected_angles = self._geometry_engine.forward_neglected_angle_terms
#     self._vacuum_reverse_neglected_angles = self._geometry_engine.reverse_neglected_angle_terms
#     self._vacuum_geometry_engine = copy.deepcopy(self._geometry_engine)
    
#     zero_state_error, one_state_error = validate_endstate_energies(forward_htf._topology_proposal, forward_htf, added_valence_energy, subtracted_valence_energy, beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD)
#     print(f"zero state error : {zero_state_error}")
#     print(f"one state error : {one_state_error}")
    
    return topology_proposal, forward_new_positions, forward_htf, local_map_stereo_sidechain, old_oemol_sidechain, new_oemol_sidechain, added_valence_energy, subtracted_valence_energy


# In[ ]:


# def create_hss(reporter_name, hybrid_factory, selection_string ='all', checkpoint_interval = 1, n_states = 13):
#     lambda_protocol = LambdaProtocol(functions='default')
#     reporter = MultiStateReporter(reporter_name, analysis_particle_indices = hybrid_factory.hybrid_topology.select(selection_string), checkpoint_interval = checkpoint_interval)
#     hss = HybridRepexSampler(mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep= 4.0 * unit.femtoseconds,
#                                                                                  collision_rate=5.0 / unit.picosecond,
#                                                                                  n_steps=250,
#                                                                                  reassign_velocities=False,
#                                                                                  n_restart_attempts=20,
#                                                                                  splitting="V R R R O R R R V",
#                                                                                  constraint_tolerance=1e-06),
#                                                                                  hybrid_factory=hybrid_factory, online_analysis_interval=10)
#     hss.setup(n_states=n_states, temperature=300*unit.kelvin,storage_file=reporter,lambda_protocol=lambda_protocol,endstates=False)
#     return hss, reporter


# let's make a function to generate an n node graph and run a computation on it...

    


# In[ ]:


def generate_fully_connected_perturbation_graph(dipeptides = ['ALA', 'PHE'], phase = 'vacuum'):
    # generate a fully connected solvation energy graph for the dipeptides specified...
    graph = nx.DiGraph()
    for dipeptide in dipeptides:
        graph.add_node(dipeptide)
    
    #now for edges...
    for i in graph.nodes():
        for j in graph.nodes():
            if i != j:
                graph.add_edge(i, j)
    
    
    #start with ala
    atp, system_generator = generate_atp(phase = phase)
    
    #graph.nodes['ALA']['vac_sys_pos_top'] = (vac_atp.system, vac_atp.positions, vac_atp.topology)
    graph.nodes['ALA']['sys_pos_top'] = (atp.system, atp.positions, atp.topology)
    
    #turn ala into all of the other dipeptides
    for dipeptide in [pep for pep in dipeptides if pep != 'ALA']:
        for _phase, testcase, sys_gen in zip([phase], [atp], [system_generator]):
            top_prop, new_positions, htf, local_map_stereo_sidechain, old_oemol, new_oemol, added_e, subtracted_e =  generate_top_pos_sys(testcase.topology, dipeptide, testcase.system, testcase.positions, sys_gen)
            new_sys, new_pos, new_top = htf._new_system, htf._new_positions, top_prop._new_topology
            graph.nodes[dipeptide][f"{_phase}_sys_pos_top"] = (new_sys, new_pos, new_top)
            graph.edges[('ALA', dipeptide)][f'{_phase}_htf'] = htf
            graph.edges[('ALA', dipeptide)][f"map_oldmol_newmol"] = (local_map_stereo_sidechain, old_oemol, new_oemol)
            graph.edges[('ALA', dipeptide)][f"added_subtracted"] = (added_e, subtracted_e)

        
        
    #now we can turn all of the other states in to each other!!!
    for edge_start, edge_end in list(graph.edges()):
        if edge_start == 'ALA': #we already did ALA
            continue
        
        for _phase, sys_gen in zip([phase], [system_generator]):
            sys, pos, top = graph.nodes[edge_start][f"{_phase}_sys_pos_top"]
            top_prop, new_positions, htf, local_map_stereo_sidechain, old_oemol, new_oemol, added_e, subtracted_e = generate_top_pos_sys(top, edge_end, sys, pos, sys_gen)
            new_sys, new_pos, new_top = htf._new_system, htf._new_positions, top_prop._new_topology
            graph.nodes[edge_end][f"{_phase}_sys_pos_top"] = (new_sys, new_pos, new_top)
            graph.edges[(edge_start, edge_end)][f"{_phase}_htf"] = htf
            graph.edges[(edge_start, edge_end)][f"map_oldmol_newmol"] = (local_map_stereo_sidechain, old_oemol, new_oemol)
            graph.edges[(edge_start, edge_end)][f"added_subtracted"] = (added_e, subtracted_e)
            
    print(f"graph_edges: {graph.edges()}")
    
    return graph
        


# In[ ]:


#os.system(f"rm *.nc")

            
        
    

    


DEBUG:numba.interpreter:label 0:
    a = arg(0, name=a)                       ['a']
    b = arg(1, name=b)                       ['b']
    $0.1 = global(np: <module 'numpy' from '/home/dominic/.local/lib/python3.6/site-packages/numpy-1.17.2-py3.6-linux-x86_64.egg/numpy/__init__.py'>) ['$0.1']
    $0.2 = getattr(value=$0.1, attr=zeros)   ['$0.1', '$0.2']
    $const0.3 = const(int, 3)                ['$const0.3']
    $0.4 = call $0.2($const0.3, func=$0.2, args=[Var($const0.3, coordinate_numba.py:6)], kws=(), vararg=None) ['$0.2', '$0.4', '$const0.3']
    c = $0.4                                 ['$0.4', 'c']
    $const0.6 = const(int, 1)                ['$const0.6']
    $0.7 = getitem(value=a, index=$const0.6) ['$0.7', '$const0.6', 'a']
    $const0.9 = const(int, 2)                ['$const0.9']
    $0.10 = getitem(value=b, index=$const0.9) ['$0.10', '$const0.9', 'b']
    $0.11 = $0.7 * $0.10                     ['$0.10', '$0.11', '$0.7']
    $const0.13 = const(int, 2)               ['$co

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


In [2]:
mapping_strength = 'default'
import pickle
phase = 'vacuum'
from perses.utils.smallmolecules import render_atom_mapping
graph = generate_fully_connected_perturbation_graph(phase = phase)
print(f"graph edges: {graph.edges()}")
# for pair in graph.edges():
#     for phase in ['vac', 'sol']:
#         print("Seralizing the system to ", f"{pair}_{phase}" + ".xml")
#         with open(f"{pair[0]}_{pair[1]}.{phase}.{mapping_strength}_map.xml", 'w') as f:
#             hybrid_system = graph.edges[pair][f"{phase}_htf"]._hybrid_system
#             f.write(openmm.openmm.XmlSerializer.serialize(hybrid_system))
        
#         htf = graph.edges[pair][f"{phase}_htf"]
#         htf._topology_proposal._old_networkx_residue.remove_oemols_from_graph()
#         htf._topology_proposal._new_networkx_residue.remove_oemols_from_graph()
#         _map, oldmol, newmol = graph.edges[pair][f"map_oldmol_newmol"]
#         render_atom_mapping(f"{pair[0]}_{pair[1]}.{mapping_strength}_map.png", oldmol, newmol, _map)
#         with open(f"{pair[0]}_{pair[1]}.{phase}.{mapping_strength}_map.pkl", 'wb') as f:
#             pickle.dump(htf, f)

INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 14
INFO:geometry:Atom index proposal order is [10, 15, 16, 18, 24, 20, 22, 25, 17, 12, 23, 11, 19, 21]
INFO:geometry:omitted_bonds: [(20, 22)]
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 16 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 54 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 82 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbon

generating point mutation engine
generating geometry engine
making topology proposal
making geometry proposal from ALA to PHE


INFO:geometry:There are 14 new atoms
INFO:geometry:	reduced angle potential = 0.08309671747062661.
INFO:geometry:	reduced angle potential = 0.15746591096172904.
INFO:geometry:	reduced angle potential = 0.33560663766135296.
INFO:geometry:	reduced angle potential = 0.1429475966355321.
INFO:geometry:	reduced angle potential = 0.028587591855980442.
INFO:geometry:	reduced angle potential = 0.380745623581824.
INFO:geometry:	reduced angle potential = 0.045349046232531526.
INFO:geometry:	reduced angle potential = 0.08503204372485398.
INFO:geometry:	reduced angle potential = 0.009109665701003015.
INFO:geometry:	reduced angle potential = 1.5462981939230205.
INFO:geometry:	reduced angle potential = 0.1923871412672404.
INFO:geometry:	reduced angle potential = 0.986532359080255.
INFO:geometry:	reduced angle potential = 0.2689035664921363.
INFO:geometry:	reduced angle potential = 1.2472758688347914.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbo

added energy components: [('CustomBondForce', 3.1945015909100904), ('CustomAngleForce', 9.80111865591279), ('CustomTorsionForce', 21.06764623455899), ('CustomBondForce', 9.74030837519539)]


INFO:geometry:log probability choice of torsions and atom order: -5.78074351579233
INFO:geometry:creating platform, integrators, and contexts; setting growth parameter
INFO:geometry:setting atoms_with_positions context old positions
INFO:geometry:There are 4 new atoms
INFO:geometry:	reduced angle potential = 0.08012155678424232.
INFO:geometry:	reduced angle potential = 7.390951941228323e-11.
INFO:geometry:	reduced angle potential = 1.2915573155343707e-10.
INFO:geometry:	reduced angle potential = 3.205828647416432e-13.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-n

added energy components: [('CustomBondForce', 0.0005201485038389051), ('CustomAngleForce', 0.4511193951899072), ('CustomTorsionForce', 7.250453392425598), ('CustomBondForce', 12.970866029941712)]
generating point mutation engine
generating geometry engine
making topology proposal
making geometry proposal from PHE to ALA


INFO:geometry:Atom index proposal order is [10, 15, 12, 11]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 36 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 42 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 98 in the reference Nonbonded 

added energy components: [('CustomBondForce', 0.5738839472034282), ('CustomAngleForce', 5.8032668764732165), ('CustomTorsionForce', 8.095543529715846), ('CustomBondForce', 12.286070947402443)]


INFO:geometry:	reduced angle potential = 0.0024118077481143753.
INFO:geometry:	reduced angle potential = 0.15746591096173265.
INFO:geometry:	reduced angle potential = 0.3356066376613585.
INFO:geometry:	reduced angle potential = 0.5718917482305678.
INFO:geometry:	reduced angle potential = 0.14294759663553555.
INFO:geometry:	reduced angle potential = 0.04534904623253347.
INFO:geometry:	reduced angle potential = 0.3807456235818296.
INFO:geometry:	reduced angle potential = 0.986532359080255.
INFO:geometry:	reduced angle potential = 1.5462981939230103.
INFO:geometry:	reduced angle potential = 0.949996361990674.
INFO:geometry:	reduced angle potential = 0.0016455783835879869.
INFO:geometry:	reduced angle potential = 0.1923871412672439.
INFO:geometry:	reduced angle potential = 0.4972236052101348.
INFO:geometry:	reduced angle potential = 0.08503204372485634.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBo

added energy components: [('CustomBondForce', 3.1945015909100904), ('CustomAngleForce', 9.80111865591279), ('CustomTorsionForce', 21.06764623455899), ('CustomBondForce', 9.74030837519539)]
graph_edges: [('ALA', 'PHE'), ('PHE', 'ALA')]
graph edges: [('ALA', 'PHE'), ('PHE', 'ALA')]


In [3]:
_dict = graph.edges['ALA', 'PHE']

In [4]:
_dict

{'vacuum_htf': <perses.annihilation.relative.HybridTopologyFactory at 0x7f114fb1de48>,
 'map_oldmol_newmol': ({},
  <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7f1117cd8fc0> >,
  <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7f1117cd8d80> >),
 'added_subtracted': (71.42274434234949, 20.672958966061046)}

In [5]:
htf = _dict['vacuum_htf']

In [6]:
#zero_state_error, one_state_error = validate_endstate_energies(_dict[f'{phase}_htf']._topology_proposal, _dict[f'{phase}_htf'], _dict['added_subtracted'][0], _dict['added_subtracted'][1], beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD)

# Hooray!!!!!!!


In [7]:
def create_new_pdb(topology, positions, output_pdb = 'test_new.pdb'):
    """
    create a pdb of the geometry proposal (only new system)
    """
    import mdtraj as md
    _positions =  np.array(positions.value_in_unit(unit.nanometer))
    print(_positions)
    traj = md.Trajectory(_positions, md.Topology.from_openmm(topology))
    traj.save(output_pdb)

In [8]:
create_new_pdb(topology = htf._topology_proposal._old_topology, positions = htf._old_positions, output_pdb = 'ALA_old.pdb')
create_new_pdb(topology = htf._topology_proposal._new_topology, positions = htf._new_positions, output_pdb = 'PHE_good.pdb')

[[ 2.0000010e-01  1.0000000e-01 -1.3000000e-07]
 [ 2.0000010e-01  2.0900000e-01  1.0000000e-08]
 [ 1.4862640e-01  2.4538490e-01  8.8982400e-02]
 [ 1.4862590e-01  2.4538520e-01 -8.8982000e-02]
 [ 3.4274200e-01  2.6407950e-01 -3.0000000e-07]
 [ 4.3905800e-01  1.8774060e-01 -6.6000000e-07]
 [ 3.5553754e-01  3.9696488e-01 -3.1000000e-07]
 [ 2.7331200e-01  4.5561601e-01 -1.3000000e-07]
 [ 4.8532621e-01  4.6139253e-01 -4.3000000e-07]
 [ 5.4075960e-01  4.3155388e-01  8.8981520e-02]
 [ 5.6613044e-01  4.2208425e-01 -1.2321480e-01]
 [ 5.1232615e-01  4.5213630e-01 -2.1312016e-01]
 [ 6.6304840e-01  4.7189354e-01 -1.2057907e-01]
 [ 5.8085401e-01  3.1408724e-01 -1.2413850e-01]
 [ 4.7126759e-01  6.1294185e-01  1.4000000e-07]
 [ 3.6006445e-01  6.6527027e-01  6.2000000e-07]
 [ 5.8460533e-01  6.8348833e-01  2.5000000e-07]
 [ 6.7370014e-01  6.3591620e-01 -4.0000000e-08]
 [ 5.8460551e-01  8.2838837e-01  6.2000000e-07]
 [ 4.8185761e-01  8.6477349e-01  1.0400000e-06]
 [ 6.3597984e-01  8.6477313e-01  8.89828

In [ ]:
import mdtraj as md

In [ ]:
top = md.Topology().from_openmm(htf._topology_proposal._new_topology)

In [ ]:
table, bonds = top.to_dataframe()

In [ ]:
table

In [ ]:
bonds